## Parser for .res files

Directory should look like this:
```
resluts
| plots.ipynb
└─final_json
   └ json files created by this notebook
└─final_res
   └ res file outputs from server testing
     should have distinct names
└─plots
   └ pdf output of plots
└─presentation
   └ complete presentation
```

In [ ]:
# Reading in data and creating a dataframe
import json
import glob
import matplotlib.pyplot as plt
import pandas as pd
from math import log
import numpy as np
from matplotlib import cm
from matplotlib.pyplot import figure
from matplotlib import colors
from matplotlib.patches import Ellipse
from math import atan

In [ ]:
for file in glob.glob('./final_res/*.res'):
    added_lines = []

    with open(file, 'r') as f:
        lines = f.readlines()
        data = {}
        for line in lines:
            
            line = line[line.index('instances/') + 10:]
            
            temp = {}
            
            if 'complex' in line:
                temp['c'] = 'complex'
            elif 'synthetic' in line:
                temp['c'] = 'synthetic'
            else:
                temp['c'] = 'unknown'
                
            if '3/' in line:
                temp['w'] = 3
            else:
                temp['w'] = 2
                
            name = line[line.index('/')+1: line.index(' ')]

            if 'correct' in line:
                split_arr = line.split('\t')
                time = split_arr[1].strip().split(' ')[0]
                rec_steps = split_arr[3].strip().split(' ')
                if len(rec_steps) < 3:
                    temp['rec'] = 0
                else:
                    temp['rec'] = int(rec_steps[2])
                temp['t'] = float(time[time.index(':')+1:]) + 60*float(time[:time.index(':')])
                temp['k'] = int(split_arr[2][1:split_arr[2].index(',')])
            else:
                if 'timelimit' not in line and 'error' in line and line[-2] == "0":
                    temp['rec'] = 0
                    temp['t'] = float(time[time.index(':')+1:]) + 60*float(time[:time.index(':')])
                    line = line[:line.index(',')]
                    line = line[line.rindex(' '):]
                    temp['k'] = int(line)
                elif 'timelimit' in line:
                    temp['rec'] = 0
                    temp['t'] = 180.0
                    temp['k'] = 0
                elif 'recursive steps' in line:
                    split_arr = line.split('\t')
                    temp['rec'] = 0
                    temp['t'] = float(time[time.index(':')+1:]) + 60*float(time[:time.index(':')])
                    temp['k'] = int(split_arr[2][1:len(split_arr[2])-1])
                else:
                    continue
                
            data[name] = temp
    with open('./final_json/' + file[12:] + '.json', 'w') as f:
        json.dump(data, f)

In [ ]:
# Merging all output json files in to one pandas DataFrame

data = {}
max_time = 180

for json_file in glob.glob('./final_json/*.json'):
    with open(json_file, 'r') as f:
        data[json_file[13:json_file.index('.res')]] = json.loads(f.read())

complete = {}
for dic in data:
    for i in data[dic]:
        if i in complete:
            continue
        else:
            complete[i] = {'k':data[dic][i]['k'],'c':data[dic][i]['c'],'w':data[dic][i]['w']}
            for j in data:
                if i in data[j]:
                    complete[i][j] = data[j][i]['t']
                else:
                    complete[i][j] = max_time

df = pd.DataFrame(complete).transpose()

In [ ]:
# completet dataframe
df

In [ ]:
# Comparison of tow models

model_1 = 'abgabe_woche03' # x-Axis
model_2 = '9combination_topo_digraph_edge_disjoint_180_sec.jar' # y-Axis

model_1_name = 'week 3'
model_2_name = 'week 4'

w = 8
h = 6

plt.figure(figsize=(w,h))

# scatter(<Model y>, <Model x>)
# complex:
plt.scatter(df[df['c'] == 'complex'][model_1], df[df['c'] == 'complex'][model_2], color= 'darkorange', marker = "+", label='complex', alpha=0.5)
# synth:
plt.scatter(df[df['c'] == 'synthetic'][model_1], df[df['c'] == 'synthetic'][model_2], color= 'darkcyan', marker = "x", label='synthetic', alpha=0.5)

plt.yscale('log')
plt.xscale('log')

# Middel line
plt.plot([10**-4, 10**3], [10**-4, 10**3], c='red', ls='--')

# Timeout Lines
plt.plot([10**-4, 10**3], [180,180], c='black', ls='--')
plt.plot([180,180], [10**-4, 10**3], c='black', ls='--')

# 10x intervalls
plt.text(0.06, 0.75, 'x10', rotation = atan(6/8)/(2*3.1415)*360)
plt.text(0.75, 0.06, 'x0.1', rotation = atan(6/8)/(2*3.1415)*360)
plt.plot([0.001,100], [0.01,1000], color='green', ls='--', lw=0.5)
plt.plot([0.01,1000], [0.001,100], color='green', ls='--', lw=0.5)

# 100x intervalls
plt.text(0.06, 7.5, 'x100', rotation = atan(6/8)/(2*3.1415)*360)
plt.text(7.5, 0.06, 'x0.01', rotation = atan(6/8)/(2*3.1415)*360)
plt.plot([0.001,10], [0.1,1000], color='green', ls='--', lw=0.5)
plt.plot([0.1,1000], [0.001,10], color='green', ls='--', lw=0.5)

# 1000x intervalls
plt.text(0.06, 75, 'x1000', rotation = atan(6/8)/(2*3.1415)*360)
plt.text(75, 0.06, 'x0.001', rotation = atan(6/8)/(2*3.1415)*360)
plt.plot([0.001,1], [1,1000], color='green', ls='--', lw=0.5)
plt.plot([1,1000], [0.001,1], color='green', ls='--', lw=0.5)

plt.xlim([5*10**-2, 300])
plt.ylim([5*10**-2, 300])
plt.title('Comparison of ' + model_1_name + ' and ' + model_2_name)
plt.xlabel(model_1_name + ' | time in seconds')
plt.ylabel(model_2_name + ' | time in seconds')
plt.legend(loc='upper left')
plt.savefig('week3_vs_week4.pdf')
plt.show()

In [ ]:
model = 'woche3'

filt = df[model] < 180

fig, ax = plt.subplots(figsize=(8,4))

complex_filt = np.logical_and(filt, df['c'] == 'complex')
synth_filt = np.logical_and(filt, df['c'] == 'synthetic')

im = ax.scatter(df[complex_filt]['n'], df[complex_filt][model], c=df[complex_filt]['k'], cmap='turbo', marker='+', s=18)
im.add_callback(ax.scatter(df[synth_filt]['n'], df[synth_filt][model], c=df[synth_filt]['k'], cmap='turbo', marker='x', s=16))

ax.set_yscale('log')
ax.set_xlabel('number of vertices')
ax.set_ylabel('time in seconds')
ax.legend(['complex', 'synthetic'])

color_bar = fig.colorbar(im, orientation="vertical")
color_bar.ax.set_ylabel('k')
plt.savefig(model + '_n-k-t-plot.pdf')
plt.show()

fig, ax = plt.subplots(figsize=(11,6))
filt = np.logical_and(filt, df['c'] == 'synthetic')


complex_filt = np.logical_and(filt, df['c'] == 'complex')
synth_filt = np.logical_and(filt, df['c'] == 'synthetic')

#im = ax.scatter(df[complex_filt]['n'], df[complex_filt][model], c=df[complex_filt]['k'], cmap='turbo', marker='+', s=18)
im = ax.scatter(df[synth_filt]['n'], df[synth_filt][model], c=df[synth_filt]['k'], cmap='turbo', marker='x', s=16)
ax.set_yscale('log')
ax.set_xlabel('number of vertices')
ax.set_ylabel('time in seconds')
ax.legend(['synthetic'])

color_bar = fig.colorbar(im, orientation="vertical")
color_bar.ax.set_ylabel('k')
plt.savefig(model + '_n-cutted_k-t-plot.pdf')
plt.show()

In [ ]:
# Scatter plot. comparing n and k with time color. Blatt 2 untere Abblidung.
model = 'woche3'

fig, ax = plt.subplots(figsize=(11,6))

filt = np.logical_and(df[model] < 180, df[model] > 0.1)
filt = np.logical_and(filt, df['c'] == 'synthetic')
#filt = np.logical_and(filt, df['n'] < 301)
#filt = np.logical_and(filt, df['k'] < 31)

im = ax.scatter(df[filt]['m']/df[filt]['n'], df[filt]['k'], c=df[filt][model], cmap='turbo',norm=colors.LogNorm(), s = 30, marker='x')
ax.set_xlabel('#edges / #vertices')
ax.set_ylabel('k')
ax.set_title('Correlation between #edges, #vertices, k and solving time in synthetic instances')
ax.add_patch(Ellipse([7.5,24], height=18, width=9, angle=-15,edgecolor='r', fc='None', lw=2, ls='--'))
color_bar = fig.colorbar(im, orientation="vertical")
color_bar.ax.set_ylabel('time in seconds')
plt.savefig('m_n_k_t_correlation.pdf');

In [ ]:
cleaning = pd.read_csv('reduction_all.csv')

c = []
w = []
for name in cleaning['instance']:
    if 'synthetic/' in name:
        c.append('s')
        w.append(2)
    elif 'synthetic3/' in name:
        c.append('s')
        w.append(3)
    elif 'complex/' in name:
        c.append('c')
        w.append(2)
    elif 'complex3/' in name:
        c.append('c')
        w.append(3)
    elif 'sheet5-heuristic/h' in name:
        c.append('h')
        w.append(5)
    elif 'sheet5-heuristic/e' in name:
        c.append('e')
        w.append(5)
cleaning['class'] = c
cleaning['week'] = w


plt.figure(figsize=(8,6))
for w in [2,3,5]:
    for c in ['s','c','h','e']:
        if w == 2:
            if c == 'c':
                m = 'x'
                colo = 'cyan'
                l = 'complex2'
            elif c == 's':
                m = '+'
                colo = 'orange'
                l = 'synthetic2'
            else:
                continue
        elif w == 3:
            if c == 'c':
                m = 'x'
                colo = 'steelblue'
                l = 'complex3'
            elif c == 's':
                m = '+'
                colo = 'red'
                l = 'synthetic3'
            else:
                continue
        elif w == 5:
            if c == 'h':
                m = '1'
                colo = 'magenta'
                l = 'pace-h'
            elif c == 'e':
                m = '2'
                colo = 'darkgreen'
                l = 'pace-e'
            else:
                continue
        else:
            continue
       
        temp = cleaning[np.logical_and(cleaning['week'] == w, cleaning['class'] == c)]
        reduction = 1 - temp['n_after']/temp['n_before']
        plt.scatter(temp['n_before'], reduction, marker = m, color=colo, label=l, alpha=0.6)
plt.xlabel('Number of vertices (n)')
plt.ylabel('Relative reduction in n')
plt.xscale('log')
plt.legend()
plt.title('Reduction in the number of vertices after full data reduction')
plt.savefig('data_reduction_all_instances.pdf')
plt.show();

cleaning.sort_values(by='time')

plt.figure(figsize=(8,6))
for w in [2,3,5]:
    for c in ['s','c','h','e']:
        if w == 2:
            if c == 'c':
                m = 'x'
                colo = 'cyan'
                l = 'complex2'
            elif c == 's':
                m = '+'
                colo = 'orange'
                l = 'synthetic2'
            else:
                continue
        elif w == 3:
            if c == 'c':
                m = 'x'
                colo = 'steelblue'
                l = 'complex3'
            elif c == 's':
                m = '+'
                colo = 'red'
                l = 'synthetic3'
            else:
                continue
        elif w == 5:
            if c == 'h':
                m = '1'
                colo = 'magenta'
                l = 'pace-h'
            elif c == 'e':
                m = '2'
                colo = 'darkgreen'
                l = 'pace-e'
            else:
                continue
        else:
            continue
        temp = cleaning[np.logical_and(cleaning['week'] == w, cleaning['class'] == c)]
        plt.scatter(temp.index, temp['time'], marker = m, color=colo, label=l, alpha=0.6)
plt.yscale('log')
plt.legend()
plt.xticks([])
plt.xlabel('Instances')
plt.ylabel('Time in seconds')
plt.title('Time required for full data reduction')
plt.savefig('data_reduction_time.pdf')
plt.show();

# Week 4 Plots

In [ ]:
df.tail(15)

In [ ]:
data = {}

for i in df.columns:
    if '_topo' in i:
        data[i] = sum(df[i] == 180) - 13

# print(data)

plt.figure(figsize=(12,8))
for ind, val in enumerate(data.values()):
    plt.bar(ind*2, val, width=0.5, color='steelblue')
    plt.annotate(xy=(ind*2, val+1), text=str(val), ha='center')
plt.xticks([i*2 for i in range(len(data))], ['\nnaive topo', 'simple reduction', '\nfull reduction', 'digraph constraints', '\n10sec', '\n20sec', '                     Adding edge-disjoint cycle constraints for up to ... sec\n30sec', '\n60sec', '\n120sec', '\n180sec'])
plt.ylabel('Number of Timeout Instances')
#plt.plot([-1, 20], [138, 138], ls='--', lw=2, c='black') solution file timeouts
plt.xlim([-0.75, 18.75])
plt.xlabel('Solver version', size=15)
plt.ylim([90,195])
plt.title('Comparisons of different solvers based on topological sorting')
plt.savefig('topo_version_comparisson.pdf')
plt.show();

## Instances solved over time

In [ ]:
model = '9combination_topo_digraph_edge_disjoint_180_sec.jar'

x_axis = np.logspace(-2, np.log10(180), 500)
x_axis = x_axis[x_axis < 180]

n = {'synthetic': {2: 441, 3: 315}, 'complex': {2: 184, 3: 137}}
y_inter = []

ax1 = plt.subplot()
for w in [2, 3]:
    for c in ['synthetic', 'complex']:
        copy_df = df.copy()
        copy_df = copy_df[copy_df['w'] == w]
        copy_df = copy_df[copy_df['c'] == c]
        y_axis = [sum(copy_df[model] < i)/n[c][w] for i in x_axis]
        plt.plot(x_axis, y_axis, label=c+str(w))
        y_inter.append(sum(copy_df[model] < 180)/n[c][w])
        
        
plt.xscale('log')
plt.legend()
plt.ylabel('Percentage of instances solved')
plt.yticks([i*2/10 for i in range(6)], [str(i*2*10) + '%' for i in range(6)])
plt.xlabel('Time in seconds')
plt.xlim([0.08, 180]);
plt.ylim([0, 1.05])

ax2 = ax1.twinx()
plt.yticks([1, 0.64, 0.936], ['100%', '65.8%', '94.4%'])
plt.ylim([0, 1.05])
plt.savefig('overtime.pdf')
plt.show();